In [1]:
#from __future__ import print_function
from DATA.lzq_read_data_time_poi import lzq_load_data
from keras.callbacks import ModelCheckpoint
#import cPickle as pickle
import numpy as np
#import math

NO=1
#for reproduction
seed=1
for i in range(NO):
    seed=seed*10+7
seed=seed*10+7
np.random.seed(seed)
#from ipdb import set_trace
#set_trace()

#for GPU in Lab
device=0

import tensorflow as tf  #from V1707
config=tf.ConfigProto()  #from V1707
config.gpu_options.allow_growth=True  #from V1707
#config.gpu_options.per_process_gpu_memory_fraction=0.15
sess=tf.Session(config=config)  #from V1707
#import keras.backend.tensorflow_backend as KTF
#KTF._set_session(tf.Session(config=config))

from keras import backend as K
K.set_image_data_format('channels_first')


#load data
epoch = 500  # number of epoch at training stage
batch_size = 32  # batch size
lr = 0.0002  # learning rate

H,W,channel = 19,21,2   # grid size

T = 24*2  # number of time intervals in one day

len_closeness = 3  # length of closeness dependent sequence
len_period = 4  # length of peroid dependent sequence

T_closeness,T_period=1,T

# last 7 days for testing data
days_test = 7
len_test = T * days_test

#the number of repetition and if retrain the model
iterate_num=5
trainDST=1  #DST
train11=1   #DSTN+ResPlus+PoI&Time
train10=1   #DSTN+ResPlus
train01=1   #DSTN+PoI&Time
train00=1   #DSTN


print("loading data...")
X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,MM=lzq_load_data(len_test,len_closeness,len_period,T_closeness,T_period)

R_N = 4   # number of residual units

from keras.optimizers import Adam
from DST_network.STResNet import stresnet
import DST_network.metrics as metrics

def build_model(external_dim,CFN):
    c_conf = (len_closeness, channel, H, W) if len_closeness > 0 else None
    p_conf = (len_period,    channel, H, W) if len_period    > 0 else None
    #t_conf = (len_trend, nb_flow, map_height,
    #          map_width) if len_trend > 0 else None

    model = stresnet(c_conf=c_conf, p_conf=p_conf, #t_conf=t_conf,
                     external_dim=external_dim, nb_residual_unit=R_N, CF=CFN)
    
    adam = Adam(lr=lr)
    model.compile(loss='mse', optimizer=adam, metrics=[metrics.rmse,metrics.mae])
    model.summary()
    #from keras.utils.visualize_util import plot
    #plot(model, to_file='model.png', show_shapes=True)
    return model

Using TensorFlow backend.


loading data...
all_data shape:  (1440, 2, 19, 21)
max= 7495.0  min= 0.0
mean= -0.8668309461018194  variance= 0.1659217996734766
number_of_skip_hours: 192


In [6]:
from keras.utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
model = build_model(False,64)
SVG(model_to_dot(model).create(prog='dot', format='svg'))

/Users/harry/Desktop/MobileBJ/DST_network/STResNet.py:69: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_size=(3, 3), filters=64, padding="same")`
  nb_filter=CF, nb_row=3, nb_col=3, border_mode="same")(input)
/Users/harry/Desktop/MobileBJ/DST_network/STResNet.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_size=(3, 3), filters=64, strides=(1, 1), padding="same")`
  return Convolution2D(nb_filter=nb_filter, nb_row=nb_row, nb_col=nb_col, subsample=subsample, border_mode="same")(activation)


TypeError: 'module' object is not callable